In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import os,errno
import sys
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import pandas as pd
import datetime as dt
%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'

#This script plots trends *season total* 95th percentile precip (no weather type subdivision)

In [ ]:
if 1==1:#part 2: read the obs at eligible stations into a better format
    yr_start=1979
    yr_end=2019
    tuples=[]
    years=[]
    months=[]
    days1=[]
    lats1=[]
    lons1=[]
    elevs1=[]
    observations=[]
    dates=[]
    station_names_list=[]
    stations_list=[]
    dp=xr.open_dataset(dir+'station_numbers_95.nc')
    stations=dp.stations.values.tolist()
    lats=dp.lats.values.tolist()
    lons=dp.lons.values.tolist()
    elevs=dp.elevs.values.tolist()
    station_names=dp.station_names.values.tolist()
    points=zip(lons,lats)
    for i in range(len(stations)):
        print(str(i+1)+'/'+str(len(stations)))
        lat=lats[i]
        lon=lons[i]
        elev=elevs[i]
        station_name=station_names[i]
        station=stations[i]
        filename=stations[i]
        ds1=pd.read_csv(filename,header=None)
        nancount=0
        qcount=0
        yescount=0
        for row1 in ds1.iterrows():
            row_neg_space=row1[1][0].replace('-',' -')
            row_basic=row1[1][0]
            a=row_neg_space.split()
            code=a[0][-4:]
            if code=='PRCP':
                year=int(a[0][11:15])
                if yr_start<=year<=yr_end:
                    month=int(a[0][15:17])
                    if month in [1,3,5,7,8,10,12]:
                        days=31
                    if month in [4,6,9,11]:
                        days=30
                    if month==2 and year%4==0:
                        days=29
                    if month==2 and year%4!=0:
                        days=28

                    for i in range(days):
                        #print(row_basic)
                        mflag=row_basic[26+8*i]
                        qflag=row_basic[27+8*i]
                        sflag=row_basic[28+8*i]
                        obs=float(row_basic[21+8*i:26+8*i])
                    
                        if obs==-9999.0:
                            pass
                        elif qflag!=' ':
                            pass
                        else:
                            years.append(year)
                            months.append(month)
                            days1.append(i)
                            observations.append(obs/10.)#convert from tenths of mm to mm
                            lats1.append(lat)
                            lons1.append(lon)
                            elevs1.append(elev)
                            date=dt.datetime(year,month,i+1,6)
                            dates.append(date)
                            station_names_list.append(station_name)
                            stations_list.append(station)
    

In [ ]:
dates_unique=list(set(dates))
dates_unique=pd.DatetimeIndex(dates_unique).sort_values()
print(dates_unique)
stations=stations_list
obs=observations
print(max(obs))

ds1=xr.open_dataset(dir+'station_numbers_95.nc')
lats_station=ds1['lats']#use these lists for plotting
lons_station=ds1['lons']
stations_station=ds1['stations']
print(stations_station)

thresholds_winter_99=ds1['thresholds_winter_99']/10.
thresholds_spring_99=ds1['thresholds_spring_99']/10.
thresholds_summer_99=ds1['thresholds_summer_99']/10.
thresholds_fall_99=ds1['thresholds_fall_99']/10.

thresholds_winter_95=ds1['thresholds_winter_95']/10.
thresholds_spring_95=ds1['thresholds_spring_95']/10.
thresholds_summer_95=ds1['thresholds_summer_95']/10.
thresholds_fall_95=ds1['thresholds_fall_95']/10.

In [ ]:
from copy import copy
import shapely.geometry as sgeom
def find_side(ls, side):
    """
    Given a shapely LineString which is assumed to be rectangular, return the
    line corresponding to a given side of the rectangle.
    
    """
    minx, miny, maxx, maxy = ls.bounds
    points = {'left': [(minx, miny), (minx, maxy)],
              'right': [(maxx, miny), (maxx, maxy)],
              'bottom': [(minx, miny), (maxx, miny)],
              'top': [(minx, maxy), (maxx, maxy)],}
    return sgeom.LineString(points[side])


def lambert_xticks(ax, ticks):
    """Draw ticks on the bottom x-axis of a Lambert Conformal projection."""
    te = lambda xy: xy[0]
    lc = lambda t, n, b: np.vstack((np.zeros(n) + t, np.linspace(b[2], b[3], n))).T
    xticks, xticklabels = _lambert_ticks(ax, ticks, 'bottom', lc, te)
    ax.xaxis.tick_bottom()
    ax.set_xticks(xticks)
    ax.set_xticklabels([ax.xaxis.get_major_formatter()(xtick) for xtick in xticklabels])
    

def lambert_yticks(ax, ticks):
    """Draw ricks on the left y-axis of a Lamber Conformal projection."""
    te = lambda xy: xy[1]
    lc = lambda t, n, b: np.vstack((np.linspace(b[0], b[1], n), np.zeros(n) + t)).T
    yticks, yticklabels = _lambert_ticks(ax, ticks, 'left', lc, te)
    ax.yaxis.tick_left()
    ax.set_yticks(yticks)
    ax.set_yticklabels([ax.yaxis.get_major_formatter()(ytick) for ytick in yticklabels])
def _lambert_ticks(ax, ticks, tick_location, line_constructor, tick_extractor):
    """Get the tick locations and labels for an axis of a Lambert Conformal projection."""
    outline_patch = sgeom.LineString(ax.outline_patch.get_path().vertices.tolist())
    axis = find_side(outline_patch, tick_location)
    n_steps = 30
    extent = ax.get_extent(ccrs.PlateCarree())
    _ticks = []
    for t in ticks:
        xy = line_constructor(t, n_steps, extent)
        proj_xyz = ax.projection.transform_points(ccrs.Geodetic(), xy[:, 0], xy[:, 1])
        xyt = proj_xyz[..., :2]
        ls = sgeom.LineString(xyt.tolist())
        locs = axis.intersection(ls)
        if not locs:
            tick = [None]
        else:
            tick = tick_extractor(locs.xy)
        _ticks.append(tick[0])
    # Remove ticks that aren't visible:    
    ticklabels = copy(ticks)
    while True:
        try:
            index = _ticks.index(None)
        except ValueError:
            break
        _ticks.pop(index)
        ticklabels.pop(index)
    return _ticks, ticklabels

In [ ]:
from scipy.stats import norm
import scipy.stats as st
def mk_test(x, alpha=0.05):
    n = len(x)

    # calculate S
    s = 0
    for k in range(n-1):
        for j in range(k+1, n):
            s += np.sign(x[j] - x[k])

    # calculate the unique data
    unique_x, tp = np.unique(x, return_counts=True)
    g = len(unique_x)

    # calculate the var(s)
    if n == g:  # there is no tie
        var_s = (n*(n-1)*(2*n+5))/18
    else:  # there are some ties in data
        var_s = (n*(n-1)*(2*n+5) - np.sum(tp*(tp-1)*(2*tp+5)))/18

    if s > 0:
        z = (s - 1)/np.sqrt(var_s)
    elif s < 0:
        z = (s + 1)/np.sqrt(var_s)
    else: # s == 0:
        z = 0

    # calculate the p_value
    p = 2*(1-norm.cdf(abs(z)))  # two tail test
    h = abs(z) > norm.ppf(1-alpha/2)

    if (z < 0) and h:
        trend = 'decreasing'
    elif (z > 0) and h:
        trend = 'increasing'
    else:
        trend = 'no trend'

    return trend, h, p, z

In [ ]:
yrs_neusa=np.arange(1979,2020,1)
zipped=list(zip(dates,stations_list,lats1,lons1,observations))
trends_list=[]
sigs_list=[]
sigs_lista=[]
print(len(zipped))
for i in range(len(stations_station)):
    print(i)
    station=stations_station[i].values
    select_station=[x for x in zipped if x[1]==station]
    threshold=thresholds_winter_95[i]
    annual_sum_list=[]
    for j in range(1,len(yrs_neusa)):
        year=yrs_neusa[j]
        if 1==0:
            select_station_year=[x for x in select_station if pd.to_datetime(x[0]).year==year and pd.to_datetime(x[0]).month in [3,4,5]]
        if 1==1:
            year_prev=yrs_neusa[j-1]
            select_station_year_prev=[x for x in select_station if pd.to_datetime(x[0]).year==year_prev and pd.to_datetime(x[0]).month==12]
            select_station_year_current=[x for x in select_station if pd.to_datetime(x[0]).year==year and pd.to_datetime(x[0]).month in [1,2]]
            select_station_year=select_station_year_prev+select_station_year_current
        
        if len(select_station_year)>0:
            select_obs=[x[-1] for x in select_station_year]
            measurable=[x for x in select_obs if x>=threshold]
            annual_sum=sum(measurable)
            annual_sum_list.append(annual_sum)
        else:
            annual_sum_list.append(0)
    annual_sum_mean=float(sum(annual_sum_list))/float(len(annual_sum_list))
    trend=mk_test(annual_sum_list,alpha=0.05)[0]
    trenda=mk_test(annual_sum_list,alpha=0.1)[0]
    slope=st.linregress(yrs_neusa[1:],annual_sum_list)[0]
    slope_percent=slope/annual_sum_mean*100.
    if trend=='increasing':
        sig=1
    elif trend=='decreasing':
        sig=-1
    else:
        sig=0
    if trenda=='increasing':
        siga=1
    elif trenda=='decreasing':
        siga=-1
    else:
        siga=0
    trends_list.append(slope_percent)
    sigs_list.append(sig)
    sigs_lista.append(siga)

In [ ]:
print(trends_list)

In [ ]:
clon=-70
clat=35
proj_map = ccrs.LambertConformal(central_longitude=clon, central_latitude=clat)
fig = plt.figure(figsize=(20,16))
ax=plt.subplot(1,1,1,projection=proj_map)

ax.coastlines(resolution='10m')
ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
ax.set_extent([-84,-66,36,48],crs=ccrs.PlateCarree())

for i in range(len(sigs_lista)):
    sig=sigs_lista[i]
    if sig in [-1,1]:#convert to mm
        ax.plot(lons_station[i],lats_station[i],transform=ccrs.PlateCarree(),marker='o',markerfacecolor='None',markeredgecolor='grey',markeredgewidth=2,markersize=28)

for i in range(len(sigs_list)):
    sig=sigs_list[i]
    if sig in [-1,1]:#convert to mm
        ax.plot(lons_station[i],lats_station[i],transform=ccrs.PlateCarree(),marker='o',markerfacecolor='None',markeredgecolor='k',markeredgewidth=2,markersize=28)
    
# *must* call draw in order to get the axis boundary used to add ticks:
fig.canvas.draw()

# Define gridline locations and draw the lines using cartopy's built-in gridliner:
xticks = [-90,-85,-80,-75,-70,-65,-60,-50]
yticks = [5,10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
ax.gridlines(xlocs=xticks, ylocs=yticks,alpha=0.5)
ax.tick_params(labelsize=26)
# Label the end-points of the gridlines using the custom tick makers:
ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
lambert_xticks(ax, xticks)
lambert_yticks(ax, yticks)
cax=ax.scatter(lons_station,lats_station,s=240,c=trends_list,transform=ccrs.PlateCarree(),vmin=-4,vmax=4,cmap=plt.cm.seismic_r,zorder=10)
cbar=plt.colorbar(cax,pad=0,extend='both',fraction=0.046)
cbar.ax.tick_params(labelsize=28)
cbar.set_label('% yr$^{-1}$',fontsize=28,rotation=90,labelpad=15)
ax.set_title('Winter',fontsize=46,pad=15)
props = dict(boxstyle='round', facecolor='wheat', alpha=1.0)
#ax.text(0.01, 0.99,'n='+str(len(trends_list)), transform=ax.transAxes, fontsize=28,verticalalignment='top', bbox=props)
plt.show()

In [ ]:
fig.savefig(dir+'climo_neusa_seasonal_trends_winter'+'.png')